In [1]:
import numpy as np
from selenium import webdriver
from time import sleep
import random
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException  
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
import re 

In [2]:
#Open URL
driver = webdriver.Chrome()
driver.get("https://www.lazada.vn/guardian/?q=All-Products&from=wangpu&langFlag=en&pageTypeId=2")
sleep(random.randint(5,10))

The chromedriver version (120.0.6099.109) detected in PATH at D:\Study\DA Practice\Project\Lazada Product Scraping\chromedriver.exe might not be compatible with the detected chrome version (129.0.6668.70); currently, chromedriver 129.0.6668.70 is recommended for chrome 129.*, so it is advised to delete the driver in PATH and retry


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 120
Current browser version is 129.0.6668.70 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
	GetHandleVerifier [0x00007FF7A4F62142+3514994]
	(No symbol) [0x00007FF7A4B80CE2]
	(No symbol) [0x00007FF7A4A276AA]
	(No symbol) [0x00007FF7A4A617E2]
	(No symbol) [0x00007FF7A4A60ADD]
	(No symbol) [0x00007FF7A4A5B9DF]
	(No symbol) [0x00007FF7A4A57F52]
	(No symbol) [0x00007FF7A4AA0AB4]
	(No symbol) [0x00007FF7A4A95D93]
	(No symbol) [0x00007FF7A4A64BDC]
	(No symbol) [0x00007FF7A4A65C64]
	GetHandleVerifier [0x00007FF7A4F8E16B+3695259]
	GetHandleVerifier [0x00007FF7A4FE6737+4057191]
	GetHandleVerifier [0x00007FF7A4FDE4E3+4023827]
	GetHandleVerifier [0x00007FF7A4CB04F9+689705]
	(No symbol) [0x00007FF7A4B8C048]
	(No symbol) [0x00007FF7A4B88044]
	(No symbol) [0x00007FF7A4B881C9]
	(No symbol) [0x00007FF7A4B788C4]
	BaseThreadInitThunk [0x00007FFA191A257D+

In [11]:
# Click the button "See more" for full of categories infomation
see_more_cat = driver.find_element("xpath", "/html/body/div[3]/div/div[3]/div[1]/div/div[2]/div/div[1]/div[2]/div/div[2]")
see_more_cat.click()

In [12]:
# GET link/title of categories
elems = driver.find_elements(By.CSS_SELECTOR , ".uM5g9 [href]")
categories = [elem.text for elem in elems]
links_cat = [elem.get_attribute('href') for elem in elems]

In [13]:
df_categories = pd.DataFrame(list(zip(categories, links_cat)), columns = ['categories','links_cat'])
pd.set_option('display.max_colwidth', None)

In [14]:
# Scraping all product data
df = pd.DataFrame()
for i in range(0, len(categories)):
    driver.get(links_cat[i])
    sleep(random.randint(5,10))
    product, links, price, total_sales, discount_all = [], [],[],[],[]
    while True:
        try:
            #GET link/title
            sleep(random.randint(6,15))
            elems = driver.find_elements(By.CSS_SELECTOR , ".RfADt [href]")
            product = [elem.text for elem in elems] + product
            links = [elem.get_attribute('href') for elem in elems]+ links
            #GET price
            elems_price = driver.find_elements(By.CSS_SELECTOR , ".aBrP0")
            price = [elem_price.text for elem_price in elems_price]+ price
            #GET total sales
            elems_sale = driver.find_elements(By.CSS_SELECTOR , "._1cEkb")
            total_sales = [elem_sale.text for elem_sale in elems_sale] + total_sales
            #GET discount
            elems_discount = driver.find_elements(By.CSS_SELECTOR , ".WNoq3")
            discount_all = [elem.text for elem in elems_discount]+ discount_all
            # next page
            try:
                sleep(random.randint(4,10))
                next_page = driver.find_element(By.CLASS_NAME,'ant-pagination-next')
                if 'ant-pagination-disabled' in next_page.get_attribute('class'):
                    print('No more pages to load')
                    break
                else:
                    next_page.click()
                    print('click to next page')
            except StaleElementReferenceException :
                break
        except NoSuchElementException:
            print('finish')
            break

    df1 = pd.DataFrame(list(zip(product, price,total_sales,discount_all, links)), columns = ['product', 'price','total_sale','discount_all','link_item'])
    df1['product_categories']= categories[i]
    df = pd.concat([df,df1], ignore_index=True)

click to next page
click to next page
No more pages to load
click to next page
click to next page
No more pages to load
click to next page
click to next page
No more pages to load
click to next page
click to next page
No more pages to load
click to next page
click to next page
No more pages to load
click to next page
No more pages to load
click to next page
No more pages to load
click to next page
No more pages to load
click to next page
No more pages to load
click to next page
No more pages to load
click to next page
No more pages to load
click to next page
No more pages to load
No more pages to load
No more pages to load
No more pages to load
No more pages to load
No more pages to load
No more pages to load
No more pages to load
No more pages to load


In [122]:
# Export to excel file
df.to_excel(r'D:\Study\DA Practice\Project\Lazada Product Scraping\lazada_product_scraping_rawdata_2.xlsx', encoding="utf-16")

In [123]:
df_categories.to_excel(r'D:\Study\DA Practice\Project\Lazada Product Scraping\product_categories_rawdata_2.xlsx', encoding="utf-16")

In [17]:
# Close browser
driver.close()

In [28]:
# Data preprocessing
df2 = pd.read_excel('lazada_product_scraping_rawdata_2.xlsx')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 825 entries, 0 to 824
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          825 non-null    int64 
 1   product             825 non-null    object
 2   price               825 non-null    object
 3   total_sale          825 non-null    object
 4   discount_all        462 non-null    object
 5   link_item           825 non-null    object
 6   product_categories  825 non-null    object
dtypes: int64(1), object(6)
memory usage: 45.2+ KB


In [45]:
df2

,Index,product,price,total_sale,discount_all,link_item,product_categories,new_price,new_total_sale,discount_raw,new_price2,new_total_sale2,discount,percent,original_price,discount_percent
0,0,Mặt Nạ Giảm Mụn Tea Tree Calming Sheet Mask So...,29.000,119,39.000 ₫\n-26%,https://www.lazada.vn/products/mat-na-giam-mun...,Mặt nạ đắp,29.000,119,39.000 ₫ -26%,29000,119,39.000,-26%,39000,26
1,1,Mặt Nạ Dừa Dưỡng Ẩm NauNau Biocell 30g/ 1 Miếng,50.000,829,NaN,https://www.lazada.vn/products/mat-na-dua-duon...,Mặt nạ đắp,50.000,829,<NA>,50000,829,<NA>,<NA>,0,0
2,2,"Mặt Nạ Dưỡng Sáng, Cấp Ẩm Banobagi Stem Cell V...",15.000,23,25.000 ₫\n-40%,https://www.lazada.vn/products/mat-na-duong-sa...,Mặt nạ đắp,15.000,23,25.000 ₫ -40%,15000,23,25.000,-40%,25000,40
3,3,Mặt Nạ Ngọc Trai Đen Dưỡng Sáng Da Platinum Co...,29.000,190,39.000 ₫\n-26%,https://www.lazada.vn/products/mat-na-ngoc-tra...,Mặt nạ đắp,29.000,190,39.000 ₫ -26%,29000,190,39.000,-26%,39000,26
4,4,[HÀNG TẶNG KHÔNG BÁN] Mặt Nạ Giấy Phục Hồi Ẩm ...,10.000.000,567,NaN,https://www.lazada.vn/products/hang-tang-khong...,Mặt nạ đắp,10.000.000,567,<NA>,10000000,567,<NA>,<NA>,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820,820,Kem Dưỡng Giảm Mụn Sau 8h La Roche-Posay Effac...,384.000,9,480.000 ₫\n-20%,https://www.lazada.vn/products/kem-duong-giam-...,Trị mụn,384.000,9,480.000 ₫ -20%,384000,9,480.000,-20%,480000,20
821,821,Kem Dưỡng Da Mụn Nhạy Cảm BIODERMA 30Ml,425.000,20,NaN,https://www.lazada.vn/products/kem-duong-da-mu...,Trị mụn,425.000,20,<NA>,425000,20,<NA>,<NA>,0,0
822,822,Gel Giảm Mụn Mờ Thâm Blissberry 15ml,139.000,474,199.000 ₫\n-30%\nMẫu thử,https://www.lazada.vn/products/gel-giam-mun-mo...,Trị mụn,139.000,474,199.000 ₫ -30% Mẫu thử,139000,474,199.000,-30% Mẫu thử,199000,30
823,823,Miếng Dán Mụn CGBIO Easyderm Band Beauty Giảm ...,60.000,15,80.000 ₫\n-25%,https://www.lazada.vn/products/mieng-dan-mun-c...,Trị mụn,60.000,15,80.000 ₫ -25%,60000,15,80.000,-25%,80000,25


In [46]:
#Cleaning data
df2['price']=df2['price'].replace(' ₫','', regex=True)
df2['total_sale']=df2['total_sale'].replace(' Đã bán','', regex=True).replace('9,999\+','10,000', regex=True)
df2.rename(columns = {'Unnamed: 0':'Index'},inplace=True)

In [47]:
df2['new_price'] = df2['price'].astype('string')
df2['new_total_sale'] = df2['total_sale'].astype('string')
df2['discount_raw'] = df2['discount_all'].astype('string')

In [48]:
df2['new_price2']=df2['new_price'].str.replace('.','', regex=True).astype('int')
df2['new_total_sale2']=df2['new_total_sale'].str.replace(',','',regex=True).astype('int')

In [49]:
df2[['discount','percent']]=df2['discount_raw'].str.split('₫',expand=True)

In [50]:
df2['original_price']=df2['discount'].str.replace('.','', regex=True).str.replace('Mẫu thử','0',regex=True).str.replace('Quà tặng','0',regex=True).replace('<N/A>','0', regex=True).fillna('0').astype('int')
df2['discount_percent']=df2['percent'].str.replace('-','',regex=True).replace('%','',regex=True).replace(' Mẫu thử','',regex=True).replace('\nMẫu thử','',regex=True).replace('\nQuà tặng','',regex=True).replace('\nLazFlash Sale, tới 23:59','',regex=True).fillna('0').astype('int')

In [51]:
df3= df2[['Index','product','new_price2','new_total_sale2','link_item','product_categories','original_price','discount_percent']]

In [52]:
df3.rename(columns = {'new_price2':'Price','new_total_sale2':'total_sale'},inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_10860\4039602053.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.rename(columns = {'new_price2':'Price','new_total_sale2':'total_sale'},inplace=True)


In [53]:
df3

,Index,product,Price,total_sale,link_item,product_categories,original_price,discount_percent
0,0,Mặt Nạ Giảm Mụn Tea Tree Calming Sheet Mask So...,29000,119,https://www.lazada.vn/products/mat-na-giam-mun...,Mặt nạ đắp,39000,26
1,1,Mặt Nạ Dừa Dưỡng Ẩm NauNau Biocell 30g/ 1 Miếng,50000,829,https://www.lazada.vn/products/mat-na-dua-duon...,Mặt nạ đắp,0,0
2,2,"Mặt Nạ Dưỡng Sáng, Cấp Ẩm Banobagi Stem Cell V...",15000,23,https://www.lazada.vn/products/mat-na-duong-sa...,Mặt nạ đắp,25000,40
3,3,Mặt Nạ Ngọc Trai Đen Dưỡng Sáng Da Platinum Co...,29000,190,https://www.lazada.vn/products/mat-na-ngoc-tra...,Mặt nạ đắp,39000,26
4,4,[HÀNG TẶNG KHÔNG BÁN] Mặt Nạ Giấy Phục Hồi Ẩm ...,10000000,567,https://www.lazada.vn/products/hang-tang-khong...,Mặt nạ đắp,0,0
...,...,...,...,...,...,...,...,...
820,820,Kem Dưỡng Giảm Mụn Sau 8h La Roche-Posay Effac...,384000,9,https://www.lazada.vn/products/kem-duong-giam-...,Trị mụn,480000,20
821,821,Kem Dưỡng Da Mụn Nhạy Cảm BIODERMA 30Ml,425000,20,https://www.lazada.vn/products/kem-duong-da-mu...,Trị mụn,0,0
822,822,Gel Giảm Mụn Mờ Thâm Blissberry 15ml,139000,474,https://www.lazada.vn/products/gel-giam-mun-mo...,Trị mụn,199000,30
823,823,Miếng Dán Mụn CGBIO Easyderm Band Beauty Giảm ...,60000,15,https://www.lazada.vn/products/mieng-dan-mun-c...,Trị mụn,80000,25


In [54]:
df3['Price'].describe()

count    8.250000e+02
mean     2.207830e+05
std      8.488485e+05
min      9.000000e+03
25%      4.700000e+04
50%      1.070000e+05
75%      2.090000e+05
max      1.000000e+07
Name: Price, dtype: float64

In [56]:
df3.loc[df3['Price'] == 10000000]

,Index,product,Price,total_sale,link_item,product_categories,original_price,discount_percent
4,4,[HÀNG TẶNG KHÔNG BÁN] Mặt Nạ Giấy Phục Hồi Ẩm ...,10000000,567,https://www.lazada.vn/products/hang-tang-khong...,Mặt nạ đắp,0,0
11,11,[HÀNG TẶNG KHÔNG BÁN] Mặt Nạ Dưỡng Da Some By ...,10000000,201,https://www.lazada.vn/products/hang-tang-khong...,Mặt nạ đắp,0,0
180,180,[HÀNG TẶNG KHÔNG BÁN] Kem Chống Nắng Dạng Tuýp...,10000000,146,https://www.lazada.vn/products/hang-tang-khong...,Kem chống nắng,0,0
237,237,[HÀNG TẶNG KHÔNG BÁN] Dầu Gội Ngăn Rụng Tóc Tr...,10000000,7,https://www.lazada.vn/products/hang-tang-khong...,Dầu gội,0,0
389,389,[HÀNG TẶNG KHÔNG BÁN] Khăn Lau Mặt Khô Guardia...,10000000,21,https://www.lazada.vn/products/hang-tang-khong...,Tẩy trang,0,0
696,696,[HÀNG TẶNG KHÔNG BÁN] Some By Mi Nước hoa hồng...,10000000,13,https://www.lazada.vn/products/hang-tang-khong...,Nước hoa hồng và Xịt khoáng,0,0


In [ ]:
#We can see from the table above that the Price column contains the highest amount of 10000000, but the average price is only 148055.8.
#This indicates that the highest amount belongs to the [[HÀNG TẶNG KHÔNG BÁN]] products, so it must be set to 0 to avoid avoid bias when calculating Revenue.

In [57]:
df3['Price']=np.where(df3['Price']==10000000,0,df3['Price'])
df3['original_price']=np.where(df3['original_price']==0,df3['Price'],df3['original_price'])
df3

C:\Users\Admin\AppData\Local\Temp\ipykernel_10860\2249432257.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Price']=np.where(df3['Price']==10000000,0,df3['Price'])
C:\Users\Admin\AppData\Local\Temp\ipykernel_10860\2249432257.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['original_price']=np.where(df3['original_price']==0,df3['Price'],df3['original_price'])


In [66]:
# Export final data to excel file
df3.to_excel(r'D:\Study\DA Practice\Project\Lazada Product Scraping\lazada_product_cleaning_data_2.xlsx', encoding="utf-16")